# Text

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.39


In [3]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [4]:
n = 25

df = pd.read_csv(f'../data/text/text_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

24


In [5]:
x_train

,text
0,It breaks my heart that this movie is not appr...
1,A Murder investigation goes on back stage whil...
2,BEFORE THE DEVIL KNOWS YOU'RE DEAD starts off ...
3,You'd think you're in for some serious sightse...
4,"I saw his movie in Dallas, Texas when it came ..."
5,This documentary on schlockmeister William Cas...
6,How I got into it: When I started watching thi...
7,This was a very good film. I didn't go into it...
8,"The movie took a new angle to Gandhi's life, w..."
9,"Ok, so it may not be the award-winning ""movie ..."


In [6]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

### Training

In [7]:
file_path = '../models/text/'

print(f'Number of data:{len(x_train)}')
model = iblm.fit(x_train, y_train, model_name = 'text', file_path=file_path)

Number of data:24
Tokens Used: 7355
	Prompt Tokens: 7087
	Completion Tokens: 268
Successful Requests: 1
Total Cost (USD): $0.22869


In [8]:
# Code of the model created
print(model)

import numpy as np
import re

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        text = row['text']
        positive_words = ['excellent', 'amazing', 'great', 'wonderful', 'best', 'love', 'good', 'favorite', 'enjoy', 'beautiful']
        negative_words = ['awful', 'terrible', 'worst', 'boring', 'disappointing', 'bad', 'hate', 'annoying', 'waste', 'ridiculous']

        positive_count = sum([1 for word in positive_words if word in text.lower()])
        negative_count = sum([1 for word in negative_words if word in text.lower()])

        # Calculate the sentiment score
        sentiment_score = (positive_count - negative_count) / (positive_count + negative_count + 1)

        # Normalize the sentiment score to a probability value between 0 and 1
        y = (sentiment_score + 1) / 2

        # Do not change the code

### Prediction

In [9]:
df = pd.read_csv(f'../data/text/text_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [10]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.7326233183856502
Precision: 0.7028914652884481
Recall: 0.8058936579115952
F1 score: 0.7508766694023727
ROC-AUC: 0.8108176450289137


### Create multiple code models

In [12]:
n = 25

df = pd.read_csv(f'../data/text/text_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']

file_path = '../models/text/'
for i in range(1,31):
    model = iblm.fit(x_train, y_train, model_name = f'text_{i}', file_path=file_path)

Tokens Used: 7352
	Prompt Tokens: 7087
	Completion Tokens: 265
Successful Requests: 1
Total Cost (USD): $0.22851
Tokens Used: 7448
	Prompt Tokens: 7087
	Completion Tokens: 361
Successful Requests: 1
Total Cost (USD): $0.23426999999999998
Tokens Used: 7456
	Prompt Tokens: 7087
	Completion Tokens: 369
Successful Requests: 1
Total Cost (USD): $0.23475
Tokens Used: 7332
	Prompt Tokens: 7087
	Completion Tokens: 245
Successful Requests: 1
Total Cost (USD): $0.22730999999999998
Tokens Used: 7353
	Prompt Tokens: 7087
	Completion Tokens: 266
Successful Requests: 1
Total Cost (USD): $0.22857
Tokens Used: 7354
	Prompt Tokens: 7087
	Completion Tokens: 267
Successful Requests: 1
Total Cost (USD): $0.22863
Tokens Used: 7424
	Prompt Tokens: 7087
	Completion Tokens: 337
Successful Requests: 1
Total Cost (USD): $0.23282999999999998
Tokens Used: 7399
	Prompt Tokens: 7087
	Completion Tokens: 312
Successful Requests: 1
Total Cost (USD): $0.23132999999999998
Tokens Used: 7463
	Prompt Tokens: 7087
	Completi

### Prediction

In [14]:
n = 25

df = pd.read_csv(f'../data/text/text_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []


for i in range(1,31):

    import_file = f'import models.text.text_{i} as codemodel'

    exec(import_file)

    try:
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(i)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(i)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            auc_list.append(roc_auc)
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(i)
        pass

print(error_count)
print(error_list)
print(auc_list)
average = sum(auc_list) / len(auc_list)
print("Average Value:", average)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

run error
run error
run error
3
[4, 7, 10]
[0.7471205861791317, 0.7381230087225826, 0.8075368240762895, 0.8127997913230525, 0.8156461272648776, 0.7993254946914317, 0.7964840353081047, 0.8080917257761557, 0.8041486707444143, 0.6774667357793527, 0.8047020655524371, 0.8156461272648776, 0.7781046229748911, 0.7381230087225826, 0.7748509837862029, 0.8251222903647978, 0.8085187564815587, 0.7471205861791317, 0.8047020655524371, 0.8127418529347432, 0.7941379586029751, 0.7983026966618254, 0.7471205861791317, 0.8465177371044941, 0.7983026966618254, 0.7943009337650883, 0.8015443897249623]
Average Value: 0.7887630503103465
Maximum Value: 0.8465177371044941
Minimum Value: 0.6774667357793527
